## How often does President Trump tweet each day?

In [1]:
import altair as alt
import pandas as pd
import matplotlib as mpl
import numpy as np
import json
import pytz
from datetime import datetime
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Get the latest 1,000 tweets from the thetrumparchive.com API endpoint

In [2]:
latest = pd.read_json('https://www.thetrumparchive.com/latest-tweets', dtype={'id':str})

In [3]:
latest.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,510761,83532,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,480997,108844,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,629326,154877,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,False,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,False,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [4]:
latest[latest['text'].str.contains('missing military ballots')]

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged


In [5]:
latest.sort_values('date', ascending=False).head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,510761,83532,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,480997,108844,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,629326,154877,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,False,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,False,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [6]:
#get latest
trump_past = pd.read_json('input/tweets_11-06-2020.json')

In [7]:
trump_extra = pd.read_json('input/tweets_11-04-2020_01-08-2021.json')

### Combine the latest 1,000 with past archive

In [8]:
src = pd.concat([latest, trump_past, trump_extra])

In [9]:
src = src.drop_duplicates(subset='text')

### Clean up dates

In [10]:
src.head()

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,isFlagged
0,1347569870578266115,"To all of those who have asked, I will not be ...",False,False,Twitter for iPhone,510761,83532,2021-01-08 15:44:28+00:00,False
1,1347555316863553542,"The 75,000,000 great American Patriots who vot...",False,False,Twitter for iPhone,480997,108844,2021-01-08 14:46:38+00:00,False
2,1347334804052844550,https://t.co/csX07ZVWGe,False,False,Twitter Media Studio,629326,154877,2021-01-08 00:10:24+00:00,False
3,1346954970910707712,These are the things and events that happen wh...,False,False,Twitter for iPhone,0,0,2021-01-06 23:01:04+00:00,False
4,1346928882595885058,https://t.co/Pm2PKV0Fp3,False,False,Twitter for iPhone,0,0,2021-01-06 21:17:24+00:00,False


In [11]:
src['eastern_created_at'] = pd.to_datetime(src['date'], utc=True)

In [12]:
src['eastern_created_at'] = pd.to_datetime(src['date'], utc=True).dt.tz_convert('America/Los_Angeles')

In [13]:
src['date'] = pd.to_datetime(src['eastern_created_at']).dt.strftime('%m/%d/%Y')
src['date'] = pd.to_datetime(src['date'])
src['year'] = src['eastern_created_at'].dt.year
src['month'] = src['eastern_created_at'].dt.month
src['day'] = src['eastern_created_at'].dt.day
src['hour'] = src['eastern_created_at'].dt.hour
src['minute'] = src['eastern_created_at'].dt.minute
src['time'] = src['eastern_created_at'].dt.time
src['id'] = src['id'].astype(str)
src['year'] = src['year'].astype(str)
src['month'] = src['month'].astype(str)
src['day'] = src['day'].astype(str)
src['hour'] = src['hour'].astype(str)

In [14]:
src.dtypes

id                                                 object
text                                               object
isRetweet                                          object
isDeleted                                          object
device                                             object
favorites                                           int64
retweets                                            int64
date                                       datetime64[ns]
isFlagged                                          object
eastern_created_at    datetime64[ns, America/Los_Angeles]
year                                               object
month                                              object
day                                                object
hour                                               object
minute                                              int64
time                                               object
dtype: object

In [15]:
november2020 = src[(src['year'] == '2020') & (src['month'] == '11')]

In [16]:
november2020.groupby(['date']).size()

date
2020-11-01    60
2020-11-02    85
2020-11-03    18
2020-11-04    14
2020-11-05    17
2020-11-06    27
2020-11-07    12
2020-11-08    19
2020-11-09    32
2020-11-10    37
2020-11-11    24
2020-11-12    48
2020-11-13    41
2020-11-14    38
2020-11-15    45
2020-11-16    41
2020-11-17    22
2020-11-18    26
2020-11-19    22
2020-11-20    24
2020-11-21    38
2020-11-22    17
2020-11-23     4
2020-11-24    41
2020-11-25    43
2020-11-26    14
2020-11-27    33
2020-11-28     8
2020-11-29    16
2020-11-30    29
dtype: int64

In [17]:
src.drop(['eastern_created_at'], axis=1, inplace=True)

In [18]:
src.reset_index().to_csv('/Users/mhustiles/data/github/notebooks/\
trump-tweets/output/realdonaldtrump.csv', index=False)